In [1]:
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import datetime

import pyodbc
from sqlalchemy import create_engine

from faker import Faker
from faker.providers import internet
import random

load_dotenv()


True

In [5]:
fake = Faker(['pt_BR'])
fake.add_provider(internet)

# from util.fake_generator import FakeGenerator
# fakegen = FakeGenerator(fake)

date_now = datetime.datetime.now()

In [9]:
file_name = 'Base BI Unificada 1.xlsx'
df = pd.read_excel(f'../../data/{file_name}')
df.head(5)

,Nome da Origem,Negócio,Grupo de Contas,Conta,Descrição da Conta,Saldo Contábil,Saldo Suporte,Diferença,Ajustes Origem,Ajustes Suporte,...,Situação Revisar,Responsável Revisar,Data Prevista Revisar,Realizador Revisar,Data Realizada Revisar,Situação Aprovar,Responsável Aprovar,Data Prevista Aprovar,Realizador Aprovar,Data Realizada Aprovar
0,2022-07,Agrichem,01. Cash,102405,Cash USD,11483936.54,11483936.54,0.0,0.0,0.0,...,Realizado,Ricardo Rodrigues Reis,2022-08-19,Euler Fernandes Rocha,2022-08-12 15:24:16.627,Realizado,Mikael Nomura,2022-08-23,Mikael Nomura,2022-08-22 16:08:26.057
1,2022-07,Agrosema (Brazil),01. Cash,102601,Cash Brazil Real,2284538.96,2284538.96,0.0,0.0,0.0,...,Pendente,Roberto Souza,2022-08-22,NaN,NaT,Pendente,Luana Estigarribio,2022-08-24,NaN,NaT
2,2022-07,Agrichem,01. Cash,102601,Cash Brazil Real,898530.09,898530.09,0.0,0.0,0.0,...,Realizado,Ricardo Rodrigues Reis,2022-08-19,Euler Fernandes Rocha,2022-08-12 15:24:02.877,Realizado,Mikael Nomura,2022-08-23,Mikael Nomura,2022-08-22 16:08:39.850
3,2022-07,APMS Participacoes Societarias,01. Cash,102601,Cash Brazil Real,91207.12,91207.12,0.0,0.0,0.0,...,Realizado,Ricardo Rodrigues Reis,2022-08-19,Fernando B Rodrigues,2022-08-19 19:22:33.150,Realizado,Pollyana Assis,2022-08-23,Pollyana Assis,2022-08-22 09:16:42.427
4,2022-07,Sementes,01. Cash,102601,Cash Brazil Real,487844.05,487844.05,0.0,0.0,0.0,...,Realizado,Ricardo Rodrigues Reis,2022-08-19,Fernando B Rodrigues,2022-08-19 19:30:01.147,Realizado,Pollyana Assis,2022-08-23,Pollyana Assis,2022-08-22 10:02:24.650


In [ ]:
qtd_linhas = len(df.index)
print(qtd_linhas)
df.columns

In [ ]:
df_purge = pd.DataFrame()

coluna_ref = 'Data'
df[coluna_ref] = pd.to_datetime(df[coluna_ref])

# df['ano'] = df[coluna_ref].dt.to_period('Y')
df_purge['ano_mes'] = df[coluna_ref].dt.to_period('M')
distribuicao_mensal = df_purge['ano_mes'].value_counts().sort_index()
distribuicao_mensal

In [ ]:
date_columns = ['Data']

for column in date_columns:
    new_column_name = 'delta_' + column
    df_purge[new_column_name] = pd.to_datetime(df[column]) - df[coluna_ref]

    media_delta = df_purge[new_column_name].mean()
    max_delta = df_purge[new_column_name].max()
    min_delta = df_purge[new_column_name].min()
    print(f'### Coluna: {column} ###')
    print(f"Média do tempo delta: {media_delta}")
    print(f"Máximo tempo delta: {max_delta}")
    print(f"Mínimo tempo delta: {min_delta} \n")

In [ ]:
df.head(5)

In [ ]:
df_faker = df.copy(deep=True)
df_faker.columns

In [16]:
random.seed(42)

In [17]:
def remove_nan(array):
    return np.array([x for x in array if x is not np.nan])

def gerar_data_fake(x, start_dt, qtd_days):
    if pd.isna(x):
        return fake.date_time_between(start_date='-1y', end_date='now')
    else:
        return x

In [23]:
seed_business = remove_nan(df_faker['Negócio'].unique())
map_business = {value: fake.company() for value in seed_business}
df_faker['Negócio'] = df_faker['Negócio'].map(map_business)


seed_account = df_faker['Conta'].unique()
map_account = {value: f"20{str(fake.unique.random_int(min=1000, max=9999))}" for value in seed_account}
df_faker['Conta'] = df_faker['Conta'].map(map_account)

seed_note = remove_nan(df_faker['Nota'].unique())
map_note = {value: fake.paragraph( variable_nb_sentences=True, ext_word_list=['abc', 'def', 'ghi', 'jkl']) for value in seed_note}
df_faker['Nota'] = df_faker['Nota'].map(map_note)

seed_resp_execute = df_faker['Responsável'].unique()
seed_real_execute = df_faker['Realizador'].unique()
seed_execute = remove_nan(pd.unique(pd.concat([pd.Series(seed_resp_execute), pd.Series(seed_real_execute)])))
map_execute = {value: fake.ascii_company_email() for value in seed_execute}
df_faker['Responsável'] = df_faker['Responsável'].map(map_execute)
df_faker['Realizador'] = df_faker['Realizador'].map(map_execute)

seed_resp_revisor = df_faker['Responsável'].unique()
seed_real_revisor = df_faker['Realizador Revar'].unique()
seed_revisor = remove_nan(pd.unique(pd.concat([pd.Series(seed_resp_revisor), pd.Series(seed_real_revisor)])))
map_revisor = {value: fake.ascii_company_email() for value in seed_revisor}
df_faker['Responsável'] = df_faker['Responsável'].map(map_revisor)
df_faker['Realizador'] = df_faker['Realizador'].map(map_revisor)

seed_resp_approve = df_faker['Responsável'].unique()
seed_resp_approve = df_faker['Realizador'].unique()
seed_approve = remove_nan(pd.unique(pd.concat([pd.Series(seed_resp_approve), pd.Series(seed_resp_approve)])))
map_approve = {value: fake.ascii_company_email() for value in seed_approve}
df_faker['Responsável'] =  df_faker['Responsável'].map(map_approve)
df_faker['Realizador'] =  df_faker['Realizador'].map(map_approve)


## Fake generate

In [24]:
data = []
for _ in range(qtd_linhas):
    id = f"10{str(fake.unique.random_int(min=1000000, max=9999999))}"

    data.append(id)

In [ ]:
df_faker['ID'] = data
df_faker.head(10)

In [28]:
year_current = date_now.year
month_current = date_now.month

df_faker = df_faker.reset_index(drop=True)
df_faker.to_excel(f'../../data/{month_current}.{year_current}.xlsx', index=None)